# Working with the Pandevice Module

What is the pandevice module?  
Back in the day, Kevin Steves created a python module called pan-python. 
http://api-lab.paloaltonetworks.com/pan-python.html

I'm not about to explain python modules but basically, the pan-python module provides a very handy way of connecting to Palo Alto devices without writing it all from scratch, and you can import this code into your script and just use it. 

Why not just use this then?  
* For one - I think it's a real pain having to know the XPATH for each thing you want to do. 
* Secondly, I'm not certain I want to put time into it when PANW could be concentrating on REST, and leaving XML API in the garden to rust.
* Lastly, a lot of other vendors provide their own python package on their github, and pandevice is that package. Not officially supported, but at least maintained.

One caveat to the above is that the pandevice module still uses pan-python to get most of it's work done. Its just that it abstracts the XPATH away and gives you objects to work on.

How does it work then?  
You import the module, create a device and tehn work with it.

In [ ]:
import pandevice
from pandevice import firewall


OK, first we need to create the object. I'm going to use getpass to hide the password (admin/admin)

In [ ]:
import getpass

In [ ]:
password = getpass.getpass('Password:')
username = input('username:')
hostname = input('hostname:')

In [ ]:
fw = pandevice.firewall.Firewall(hostname, username, password)


In [ ]:
fw

In [ ]:
dir(fw)

OK, so those are the items and methods of fw. Notice the commit() method - handy! 

In [ ]:
sysinf = fw.refresh_system_info()

In [ ]:
print(f'Updated system info from {fw.hostname} is : {sysinf}')

OK - so we've used the module to connect to the API, but at this stage, its just like having a CLI window open. You can't see the config until you get it. Pandevice works by letting you gra the whole config, or just getting parts of it and working on them. Alternatively, you can create your own bits and add them to the firewall.

We can use the device, network, policies, panorama or object modules to create sections. Here, we will use the device module to get system settings.  

We can just get the settings, or actually add them to our local version and work on them.

In [ ]:
settings = pandevice.device.SystemSettings.refreshall(fw, add=True)

In [ ]:
settings

OK, that seemed to work but can I actually see anything? Use dir() to check options. (it's like ? on CLI)

In [ ]:
dir(settings)

In [ ]:
print(type(settings))

OK, so you can see it's a list, so let's have a look into it by listing the contents [:]

In [ ]:
settings[:]

OK, a list with only one entry. Right, now we're getting somewhere. What type of thing is this?

In [ ]:
print(type(settings[0]))

In [ ]:
dir(settings[0])

OK, the about() method looks good for find out info. Also, there was an apply() method there.

In [ ]:
settings[0].about()

Let's change something then.

In [ ]:
settings[0].domain = 'mynewdomain.local'

In [ ]:
settings[0].domain

So now our local copy is updated, we can push to the firewall. 

In [ ]:
settings[0].apply()

And if that worked, maybe we can commit.

In [ ]:
status = fw.commit()
print(f'job number {status} complete')

OK - so that should have commited on job 3 and set the domain. Let's check on the GUI.

SO let's have a look at putting that together in a script. examples/pandriver-demo.py